In [1]:
import time

import torch
from torch.utils.data import DataLoader

from torchtext import datasets
from torchtext.data import Field
from torchtext.data.utils import get_tokenizer

from transformers import AutoTokenizer

from utils import extract_config
from constants import *

import lineflow.datasets as lfds
from transformer_pl import *
from data_pl import *

# config
config = {
    "embedding_dimension": 200,
    "ff_dimension": 200,
    "n_attention_heads": 2,
    "n_encoder_layers": 0,
    "n_decoder_layers": 2,
    "dataset": Dataset.PennTreebank.name,
    "segmentation": Segmentation.Word.name,
    "max_seq_len": 35,
    "batch_size": 20,
    "eval_batch_size": 10,
    "dropout": 0.2,
    "n_epochs": 3,
    "learning_rate": 0.0001,
    "adam_b1": 0.9,
    "adam_b2": 0.999,
    "adam_l2_weightdecay": 0.01,
    "loss_criterion": "CrossEntropyLoss"
}

# setup
train_dataloader, val_dataloader, test_dataloader, vocab = load_data_word(config)
ntokens = len(vocab.stoi)

# init model
model = DecoderOnlyTransformer(config, ntokens)
trainer = pl.Trainer(gpus=2, accelerator="dp")
trainer.fit(model, train_dataloader, val_dataloader)

[Start Load Data]
Fetched Data (0.000012s)
Tokenized and Split Data (0.685161s)
Built Vocab (0.850403s)


TypeError: __init__() missing 1 required positional argument: 'max_seq_len'

In [3]:
train_dataloader